# Lista 4, Métodos Numéricos 2 
Javier Alejandro Ovalle Chiquín, 22103  
Ricardo Josué Morales Contreras, 22289

#### Problema 1. Modelo de Prooducción, Periodo único

Solución del modelo con la librería pulp sin restricción de enteros.

In [2]:
import pulp

# Crear el problema
prob = pulp.LpProblem("Produccion_Optima", pulp.LpMaximize)

# Variables continuas
x1 = pulp.LpVariable("Abrigos", lowBound=0)
x2 = pulp.LpVariable("Chamarras", lowBound=0)
x3 = pulp.LpVariable("Pantalones", lowBound=0)
x4 = pulp.LpVariable("Guantes", lowBound=0)
s1 = pulp.LpVariable("Faltante_Abrigos", lowBound=0)
s2 = pulp.LpVariable("Faltante_Chamarras", lowBound=0)
s3 = pulp.LpVariable("Faltante_Pantalones", lowBound=0)
s4 = pulp.LpVariable("Faltante_Guantes", lowBound=0)

# Objetivo
prob += 30*x1 + 40*x2 + 20*x3 + 10*x4 - 15*s1 - 20*s2 - 10*s3 - 8*s4

# Restricciones de demanda
prob += x1 + s1 == 800
prob += x2 + s2 == 750
prob += x3 + s3 == 600
prob += x4 + s4 == 500

# Restricciones de departamentos
prob += 0.30*x1 + 0.30*x2 + 0.25*x3 + 0.15*x4 <= 1000, "Corte"
prob += 0.25*x1 + 0.35*x2 + 0.30*x3 + 0.10*x4 <= 1000, "Aislamiento"
prob += 0.45*x1 + 0.50*x2 + 0.40*x3 + 0.22*x4 <= 1000, "Costura"
prob += 0.15*x1 + 0.15*x2 + 0.10*x3 + 0.05*x4 <= 1000, "Empaque"

# Resolver
prob.solve(pulp.PULP_CBC_CMD(msg=0))

# Resultados
print("Valor objetivo:", pulp.value(prob.objective))
print("x1:", x1.varValue)
print("x2:", x2.varValue)
print("x3:", x3.varValue)
print("x4:", x4.varValue)
print("s1:", s1.varValue)
print("s2:", s2.varValue)
print("s3:", s3.varValue)
print("s4:", s4.varValue)

Valor objetivo: 64625.0
x1: 800.0
x2: 750.0
x3: 387.5
x4: 500.0
s1: 0.0
s2: 0.0
s3: 212.5
s4: 0.0


Estos resultados coinciden exáctamente con los mostrados por simplex de excel

Solución del problema, añadiendo la restricción de que las variables sean enteras:

In [3]:
import pulp

# Crear el problema
prob = pulp.LpProblem("Produccion_Entera", pulp.LpMaximize)

# Variables enteras
x1 = pulp.LpVariable("Abrigos", lowBound=0, cat='Integer')
x2 = pulp.LpVariable("Chamarras", lowBound=0, cat='Integer')
x3 = pulp.LpVariable("Pantalones", lowBound=0, cat='Integer')
x4 = pulp.LpVariable("Guantes", lowBound=0, cat='Integer')
s1 = pulp.LpVariable("Faltante_Abrigos", lowBound=0, cat='Integer')
s2 = pulp.LpVariable("Faltante_Chamarras", lowBound=0, cat='Integer')
s3 = pulp.LpVariable("Faltante_Pantalones", lowBound=0, cat='Integer')
s4 = pulp.LpVariable("Faltante_Guantes", lowBound=0, cat='Integer')

# Objetivo 
prob += 30*x1 + 40*x2 + 20*x3 + 10*x4 - 15*s1 - 20*s2 - 10*s3 - 8*s4

# Restricciones 
prob += x1 + s1 == 800
prob += x2 + s2 == 750
prob += x3 + s3 == 600
prob += x4 + s4 == 500
prob += 0.30*x1 + 0.30*x2 + 0.25*x3 + 0.15*x4 <= 1000, "Corte"
prob += 0.25*x1 + 0.35*x2 + 0.30*x3 + 0.10*x4 <= 1000, "Aislamiento"
prob += 0.45*x1 + 0.50*x2 + 0.40*x3 + 0.22*x4 <= 1000, "Costura"
prob += 0.15*x1 + 0.15*x2 + 0.10*x3 + 0.05*x4 <= 1000, "Empaque"

# Resolver
prob.solve(pulp.PULP_CBC_CMD(msg=0))

# Resultados
print("Valor objetivo:", pulp.value(prob.objective))
print("x1:", x1.varValue)
print("x2:", x2.varValue)
print("x3:", x3.varValue)
print("x4:", x4.varValue)
print("s1:", s1.varValue)
print("s2:", s2.varValue)
print("s3:", s3.varValue)
print("s4:", s4.varValue)

Valor objetivo: 64622.0
x1: 800.0
x2: 750.0
x3: 388.0
x4: 499.0
s1: 0.0
s2: 0.0
s3: 212.0
s4: 1.0


Como se puede observar varian los resultados en $x_3$ que son el número de pantalones producidos, ya que era la solución que no quedaba entera, modificandolo a que sean 388 pantalones y restanto 1 al número de guantes producidos teniendo $212$ pantalones faltantes y 1 guante faltante llegando a la solución optima de $64622$. 

Lo interesante viene al compararlo con el método simplex de excel, ya que la solución optima fue dejando $387$ pantalones y $213$ faltantes, llegando a una subsolucion de $64610$, la cual es menos a la solución propuesta por pulp. Esto probablemente se debe a las limitantes de las interaciones con excel.

#### Problema 2. Modelo de Producción, periodos múltiples 

In [4]:
import pulp

# Datos
demandas = [180, 250, 190, 140, 220, 250]
costos_prod = [50, 45, 55, 52, 48, 50]
costos_inv = [8, 10, 10, 10, 8, 8]
capacidad = 225
meses = range(1,7)

# Problema continuo
prob_cont = pulp.LpProblem("Produccion_Multiperiodo_Cont", pulp.LpMinimize)

P = pulp.LpVariable.dicts("Produccion", meses, lowBound=0, upBound=capacidad)
I0 = pulp.LpVariable("I0", lowBound=0, upBound=0)
I = pulp.LpVariable.dicts("Inventario", meses, lowBound=0)
I6 = pulp.LpVariable("I6", lowBound=0)

# Objetivo
prob_cont += pulp.lpSum(costos_prod[t-1]*P[t] + costos_inv[t-1]*I[t] for t in meses)

# Balance
prev_I = I0
for t in meses:
    if t < 6:
        prob_cont += prev_I + P[t] == demandas[t-1] + I[t]
        prev_I = I[t]
    else:
        prob_cont += prev_I + P[t] == demandas[t-1] + I6
prob_cont += I6 == 0

prob_cont.solve(pulp.PULP_CBC_CMD(msg=0))

# Resultados
print("Objetivo:", pulp.value(prob_cont.objective))
for t in meses:
    print(f"Mes {t}: P={P[t].varValue:.2f}, I_end={I[t].varValue:.2f}")

Objetivo: 61795.0
Mes 1: P=205.00, I_end=25.00
Mes 2: P=225.00, I_end=0.00
Mes 3: P=190.00, I_end=0.00
Mes 4: P=160.00, I_end=20.00
Mes 5: P=225.00, I_end=25.00
Mes 6: P=225.00, I_end=0.00


La estrategia óptima produce al máximo (225) en meses baratos (2 y 5, con $ c_2=45 $, $ c_5=48 $) para cubrir parte de las demandas altas en 2 y 6, almacenando el excedente. En mes 1, produce extra (205 > 180) para ayudar en mes 2. En mes 4, produce 160 > 140 pero almacena para mes 5 y 6. Mes 3 produce exacto ya que $ c_3=55 $ es alto.

Para la comparativa con la solución no óptima, que es producir exactamente la cantidad de la demanda por cada mes se observa a simple vista que no se llegará a cumplir la demanda de los meses 2 y 6 ya que la capacidad máxima de producir es de 225 mientras que la demanda será de 250 en ambos meses, sin embargo se obtiene una solución la cual sería calculada por $180 \cdot 50+ 225 \cdot 45 + 190 \cdot 55 + 140 \cdot 52 + 220 \cdot 48 + 225 \cdot 50 = 58,665 $ sin llegar a la demanda por la falta de capacidad de producción.
